# Waves And Markers Download Test

In [1]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
import pyte_visa_utils as pyte

In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::172.16.10.2::5025::SOCKET'

print('Connecting ..')
inst = pyte.open_session(inst_addr)
resp = inst.query("*IDN?")
print('Connected to: ' + resp)


Connecting ..
Connected to: Tabor Electronics,P2584M,000000220725,1.115.0


In [4]:
# The paranoia-level is either 
# low (0), normal (1) or debug (2)
paranoia_level = 2

print ('Reset instrument ..')

# Reset the instrument
pyte.send_cmd(inst, '*CLS; *RST', paranoia_level)

print('Done')

Reset instrument ..
Done


In [5]:
# Get the DAC mode (8 bits or 16 bits)
resp = inst.query(':SYST:INF:DAC?')
dac_mode = resp
print('DAC mode {0} '.format(dac_mode))

if dac_mode == 'M0':
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac // 2.0

# Get number of channels
resp = inst.query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))



DAC mode M0 
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 4,286,578,688 wave-bytes


## Prepare wave-data and markers-data for 12 segments


In [6]:
# Build wave-data and markers-data for 12 segments

print('Build wave-data and markers-data for 12 segments ..')

seglen = 2 ** 12 # 1MB
num_cycles = [2 ** n for n in range(12)]

waves = [ None for _ in range(12)]
marks = [ None for _ in range(12)]

if dac_mode == 16:
    seg_wave_bytes = seglen * 2
else:
    seg_wave_bytes = seglen

seg_mark_bytes = seg_wave_bytes // 8

for ii in range(12):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    
    
    if dac_mode == 16:
        cycle_bytes = cyclelen // 4
    else:
        cycle_bytes = cyclelen // 8
        
    x = np.linspace(
        start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
    y = np.fmod(x, cycle_bytes)
    y = (y <= cycle_bytes / 2) * 255
    y = np.round(y)
    y = np.clip(y, 0, 255)
    marks[ii] = y.astype(np.uint8)
    
    del x, y
    
print('Done')   
 

Build wave-data and markers-data for 12 segments ..
Done


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in fmod
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in less_equal


## Download wave-data and markers-data to segments

In [7]:
# Download segments

# data-type code: 'B' = uint8, 'H' = uint16
wav_dat_type = 'H' if dac_mode == 16 else 'B'
mrk_dat_type = 'B'

for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 1
    wav = waves[ichan]
    mrk = marks[ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Send the binary-data with *OPC? added to the beginning of its prefix.
    inst.write_binary_values('*OPC?; :TRAC:DATA', wav, datatype=wav_dat_type)
    
    # Read the response to the *OPC? query that was added to the prefix of the binary data
    resp = inst.read()
    
    # Set normal timeout
    inst.timeout = 10000
    
    resp = inst.query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))
        
    print('Download markers to segment {0} of channel {1}'.format(segnum, channb))
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Send the binary-data with *OPC? added to the beginning of its prefix.
    inst.write_binary_values('*OPC?; :MARK:DATA', mrk, datatype=mrk_dat_type)
    
    # Read the response to the *OPC? query that was added to the prefix of the binary data
    resp = inst.read()
    
    # Set normal timeout
    inst.timeout = 10000
    
    resp = inst.query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)

    # Turn on the output of the selected channel:
    cmd = ':OUTP ON'
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Turn on the markers of the selected channel    
    resp = inst.query(':MARK:SEL? MAX')
    resp = resp.rstrip()
    markers_per_chan = int(resp)
    for imarker in range(markers_per_chan):
        cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker + 1)
        pyte.send_cmd(inst, cmd, paranoia_level)
    
    resp = inst.query(':SYST:ERR?')
    print(resp)

print()

resp = inst.query(':SYST:ERR?')
print(resp)

del waves
del wav

Download wave to segment 1 of channel 1
Download markers to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
Download markers to segment 2 of channel 2
0, no error
Download wave to segment 1 of channel 3
Download markers to segment 1 of channel 3
0, no error
Download wave to segment 2 of channel 4
Download markers to segment 2 of channel 4
0, no error

0, no error


## Transfer files of wave and markers data to instrument's file-system

In [8]:
print ('Reset instrument and delete all segment definitions ..')

# Reset the instrument
pyte.send_cmd(inst, '*CLS; *RST', paranoia_level)

# Delete all segment definitions in the segment-table of the selected channel
pyte.send_cmd(inst, ':TRACe:DELete:ALL', paranoia_level)

# The destination file-name in the instrument's file-system
wave_file_name = 'wave_file_test.wav'

# The destination file-name in the instrument's file-system
mark_file_name = 'mark_file_test.dat'

print('Done')

Reset instrument and delete all segment definitions ..
Done


###  Prepare wave-data and save it in file in the instrument's file-system 

#### 1. Prepare wave data

In [9]:
wavdatsize = 64 * (2**20) # 64 MB

if dac_mode == 16:
    bpp = 2
    wavdattype = 'H'
else:
    bpp = 1
    wavdattype = 'B'

wavdatlen = wavdatsize // bpp
    
ncycles = 64

print ('Preparing wave data ..')
x = np.linspace(
        start=0, stop=2 * np.pi * ncycles, num=wavdatlen, endpoint=False)
wavdat = (np.sin(x) + 1.0) * half_dac
wavdat = np.round(wavdat)
wavdat = np.clip(wavdat, 0, max_dac)
if dac_mode == 16:
    wavdat = wavdat.astype(np.uint16)
else:
    wavdat = wavdat.astype(np.uint8)

print('Done ({0} wave points of type {1})'.format(len(wavdat), wavdat.dtype))

Preparing wave data ..
Done (67108864 wave points of type uint8)


#### 2. Send the destination wave-file name as binary-data

In [10]:
# Set the path of the destination wave-file.
# The file-path is sent as binary data with ':SYSTem:FILE:NAME'.
# If the path is just file-name then the file is put in 'Documents/ProteusFiles'

# Convert the path-name to byte-array
wave_file_name = wave_file_name.encode('ascii')


print ('Downloading file-name as binary data ..')

# Send it as binary data
inst.timeout = 30000
inst.write_binary_values('*OPC?; :SYST:FILE:NAME', wave_file_name, datatype='B')
resp = inst.read()
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

0, no error


####  3. Write the wave-data in the destination-file

In [11]:
# Write the data to the destination file using 
# ':SYSTem:FILE:DATA <offset-in-bytes>#<binary-header><binary-block>'

chunksize = 2 ** 20 # 1MB
chunklen = chunksize // bpp

offset_in_bytes = 0

print ('Downloading wave data to file ..')

inst.timeout = 30000
while offset_in_bytes < wavdatsize:
    
    if wavdatsize - offset_in_bytes < chunksize:
        chunksize = wavdatsize - offset_in_bytes
        chunklen = chunksize // bpp
    
    cmd = '*OPC?; :SYST:FILE:DATA {0}'.format(offset_in_bytes)
    
    offs_in_pts = offset_in_bytes // bpp
    chunk = wavdat[offs_in_pts: offs_in_pts + chunklen]   
    
    inst.write_binary_values(cmd, chunk, datatype=wavdattype)
    resp = inst.read()    
    
    offset_in_bytes = offset_in_bytes + chunksize
    
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

del wavdat

0, no error


### Prepare markers-data and save it in file in the instrument's file-system

#### 1. Prepare markers data

In [12]:
print ('Preparing markers data ..')

mrkdatsize = wavdatsize // 8
mark_cycle_size = mrkdatsize // ncycles
mrkdat = np.zeros(mrkdatsize, dtype=np.uint8)

for icycle in range(ncycles):
    n = icycle * mark_cycle_size
    m = n + mark_cycle_size // 2
    mrkdat[n:m] = 255
    
print ('Done ({0} bytes of markers data)'.format(len(mrkdat)))
   

Preparing markers data ..
Done (8388608 bytes of markers data)


#### 2. Send the destination markers-file name as binary data

In [13]:
# Set the path of the destination markers file
# The file-path is sent as binary data with ':SYSTem:FILE:NAME'.
# If the path is just file-name then the file is put in 'Documents/ProteusFiles'


# Convert the path-name to byte-array
mark_file_name = mark_file_name.encode('ascii')

print ('Downloading file-name as binary data ..')

# Send it as binary data
inst.timeout = 30000
inst.write_binary_values('*OPC?; :SYST:FILE:NAME', mark_file_name, datatype='B')
resp = inst.read()
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

0, no error


####  3. Write the markers-data in the destination-file

In [14]:
print ('Downloading markers data to file ..')

offset_in_bytes = 0
chunksize = 2 ** 20 # 1MB

inst.timeout = 30000

while offset_in_bytes < mrkdatsize:
    
    if mrkdatsize - offset_in_bytes < chunksize:
        chunksize = mrkdatsize - offset_in_bytes
    
    cmd = '*OPC?; :SYST:FILE:DATA {0}'.format(offset_in_bytes)
    
    chunk = mrkdat[offset_in_bytes: offset_in_bytes + chunksize]    
    
    inst.write_binary_values(cmd, chunk, datatype='B')
    resp = inst.read()    
    
    offset_in_bytes = offset_in_bytes + chunksize
    
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

del mrkdat

0, no error


### List files in the instrument's file-system

In [15]:
# Get list of all files in 'Documents/ProteusFile'

print('Query files list ..')
resp = inst.query(':SYST:FILE:CATalog?')
print(resp)


# Query the size of the wave-data file and the markers-data file:

for filename in (wave_file_name, mark_file_name):    
    print()
    print('Query the size of {0} ..'.format(filename))

    # Set file name:    
    inst.timeout = 30000
    inst.write_binary_values('*OPC?; :SYST:FILE:NAME', filename, datatype='B')
    resp = inst.read()
    inst.timeout = 10000

    resp = inst.query(':SYST:ERR?')
    if not resp.startswith('0'):
        print('Error "{0}" after ":SYST:FILE:NAME"'.format(resp.strip()))

    # Query file size
    resp = inst.query(':SYST:FILE:SIZE?')
    numbytes = int(resp.strip())
    print('The file size is {0} bytes'.format(numbytes))

Query files list ..
'mark_file_test.dat','wave_file_test.wav'

Query the size of b'wave_file_test.wav' ..
The file size is 67108864 bytes

Query the size of b'mark_file_test.dat' ..
The file size is 8388608 bytes


### Load wave-data and markers-data  from file to segment 

#### 1. Define segment

In [16]:
channb = 1
segnum = 1
seglen = wavdatlen

print('Define segment {0} of {1} wave-points in channel {2} ..'.format(segnum, seglen, channb))

# Select channel
cmd = ':INST:CHAN {0}'.format(channb)
pyte.send_cmd(inst, cmd, paranoia_level)

# Define segment
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
pyte.send_cmd(inst, cmd, paranoia_level)

resp = inst.query(':SYST:ERR?')
print(resp)

Define segment 1 of 67108864 wave-points in channel 1 ..
0, no error


#### 2. Load wave data to segment from file

In [17]:
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
pyte.send_cmd(inst, cmd, paranoia_level)

# -----------------------------------
# Load wave-data to segment from file
# -----------------------------------

# First set the source-file path using ':TRACE:FILE[:NAME]'
# If the path is just file-name the the default directory ('Documents/ProteusFiles') is assumed

print('Set source wave-file path ..')

# Send it as binary data
inst.timeout = 30000
inst.write_binary_values('*OPC?; :TRAC:FILE:NAME', wave_file_name, datatype='B')
resp = inst.read()
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

# Load data from file to segment with ':TRAC:FILE:LOAD' command.
# Without any arguments it shall load the whole file from the beginning of the segment

print('Loading wave-data to segment from file ..')

# If you send the command as query, make sure you increase the timeout ..

inst.timeout = 30000
pyte.send_cmd(inst, ':TRAC:FILE:LOAD', paranoia_level)
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

Set source wave-file path ..
0, no error
Loading wave-data to segment from file ..
0, no error


#### 3. Load markers data to segment from file

In [18]:
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
pyte.send_cmd(inst, cmd, paranoia_level)

# --------------------------------------
# Load markers-data to segment from file
# --------------------------------------

# First set the source-file path using ':MARK:FILE[:NAME]'
# If the path is just file-name the the default directory ('Documents/ProteusFiles') is assumed

print('Set source marker-file path ..')

# Send it as binary data
inst.timeout = 30000
inst.write_binary_values('*OPC?; :MARK:FILE:NAME', mark_file_name, datatype='B')
resp = inst.read()
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

# Load data from file to segment with ':MARK:FILE:LOAD' command.
# Without any arguments it shall load the whole file from the beginning of the segment

print('Loading markers-data to segment from file ..')

# If you send the command as query, make sure you increase the timeout ..

inst.timeout = 3000000
pyte.send_cmd(inst, ':MARK:FILE:LOAD', paranoia_level)
inst.timeout = 10000

resp = inst.query(':SYST:ERR?')
print(resp)

Set source marker-file path ..
0, no error
Loading markers-data to segment from file ..
0, no error


#### 4. Play the segment 

In [19]:
print('Play the segment ..')

# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
pyte.send_cmd(inst, cmd, paranoia_level)

# Turn on the output of the selected channel:
cmd = ':OUTP ON'
pyte.send_cmd(inst, cmd, paranoia_level)

# Turn on the markers of the selected channel    
resp = inst.query(':MARK:SEL? MAX')
resp = resp.rstrip()
markers_per_chan = int(resp)
for imarker in range(markers_per_chan):
    cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker + 1)
    pyte.send_cmd(inst, cmd, paranoia_level)

resp = inst.query(':SYST:ERR?')
print(resp)

Play the segment ..
0, no error


### Delete files from the instrument's file-system

In [20]:
for filename in (wave_file_name, mark_file_name):    

    print('Delete "{0}" ..'.format(filename))

    # Set file name:    
    inst.timeout = 30000
    inst.write_binary_values('*OPC?; :SYST:FILE:NAME', filename, datatype='B')
    resp = inst.read()
    inst.timeout = 10000

    resp = inst.query(':SYST:ERR?')
    if not resp.startswith('0'):
        print('Error "{0}" after ":SYST:FILE:NAME"'.format(resp.strip()))

    # Delete file:
    pyte.send_cmd(inst, ':SYST:FILE:DELete', paranoia_level)
    resp = inst.query(':SYST:ERR?')
    print(resp)

Delete "b'wave_file_test.wav'" ..
0, no error
Delete "b'mark_file_test.dat'" ..
0, no error
